# TIỀN XỬ LÝ DỮ LIỆU AIRBNB NYC 2019

Notebook này thực hiện tiền xử lý dữ liệu để chuẩn bị cho việc training model.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.append("../src")
from data_processing import (load_data, handle_missing_values, normalize_data, 
                             get_numeric_columns, save_data)
from visualization import plot_comparison

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Load Dữ liệu

In [7]:
data_path = '../data/raw/AB_NYC_2019.csv'
data, headers = load_data(data_path)

print(f"Kích thước dữ liệu gốc: {data.shape}")
print(f"Các cột: {', '.join(headers)}")

Kích thước dữ liệu gốc: (48895, 16)
Các cột: id, name, host_id, host_name, neighbourhood_group, neighbourhood, latitude, longitude, room_type, price, minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count, availability_365


### Ý nghĩa của việc xử lý Missing Values:

**Quyết định:**
- Loại bỏ các hàng có missing values quan trọng (price, location)
- Điền 0 cho reviews_per_month (chưa có review)
- Điền median cho các cột số khác

**Lý do:**
- Price và location là thông tin bắt buộc để dự đoán
- Reviews_per_month = 0 có nghĩa là listing mới chưa có đánh giá
- Median tránh bị ảnh hưởng bởi outliers so với mean

**Ảnh hưởng:**
- Giảm bias trong mô hình
- Giữ lại nhiều dữ liệu hơn so với việc xóa tất cả missing rows
- Đảm bảo chất lượng dữ liệu cho training

## 2. Xử lý Missing Values

In [8]:
print("Missing values trước khi xử lý:")
for idx, header in enumerate(headers):
    col = data[:, idx]
    missing_count = np.sum((col == '') | (col == None))
    if missing_count > 0:
        print(f"  {header}: {missing_count}")

Missing values trước khi xử lý:
  name: 16
  host_name: 21
  last_review: 10052
  reviews_per_month: 10052


In [9]:
data_cleaned = handle_missing_values(data, headers, strategy='median')

print("\nMissing values sau khi xử lý:")
for idx, header in enumerate(headers):
    col = data_cleaned[:, idx]
    missing_count = np.sum((col == '') | (col == None))
    if missing_count > 0:
        print(f"  {header}: {missing_count}")


Missing values sau khi xử lý:
  name: 16
  host_name: 21
  last_review: 10052


## 3. Xử lý Outliers

In [10]:
price_idx = headers.index('price')
price_col = data_cleaned[:, price_idx].astype(float)

print(f"Số lượng records ban đầu: {len(data_cleaned)}")
print(f"Price - Min: {np.min(price_col):.2f}, Max: {np.max(price_col):.2f}")

valid_price_mask = (price_col > 0) & (price_col <= 1000)
data_filtered = data_cleaned[valid_price_mask]

print(f"\nSố lượng records sau khi lọc price: {len(data_filtered)}")
print(f"Đã loại bỏ: {len(data_cleaned) - len(data_filtered)} records")

Số lượng records ban đầu: 48895
Price - Min: 0.00, Max: 10000.00

Số lượng records sau khi lọc price: 48645
Đã loại bỏ: 250 records


### Lý do loại bỏ Outliers:

**Chiến lược:**
- Loại bỏ price > $1000 và minimum_nights > 365

**Giải thích:**
- Giá quá cao (>$1000) là các trường hợp đặc biệt, không đại diện cho thị trường chung
- Minimum nights > 365 ngày có thể là lỗi nhập liệu hoặc cho thuê dài hạn
- Các outliers này làm méo mô hình, dẫn đến dự đoán kém chính xác

**Kết quả:**
- Mô hình sẽ chính xác hơn cho phân khúc thị trường chính (80-90% listings)
- Trade-off: Không dự đoán tốt cho luxury listings hoặc long-term rentals

In [20]:
min_nights_idx = headers.index('minimum_nights')
min_nights_col = data_filtered[:, min_nights_idx].astype(float)

print(f"Minimum_nights - Min: {np.min(min_nights_col):.2f}, Max: {np.max(min_nights_col):.2f}")

valid_nights_mask = min_nights_col <= 365
data_filtered = data_filtered[valid_nights_mask]

print(f"Số lượng records sau khi lọc minimum_nights: {len(data_filtered)}")

Minimum_nights - Min: 1.00, Max: 365.00
Số lượng records sau khi lọc minimum_nights: 48631


## 4. Feature Engineering

In [12]:
reviews_idx = headers.index('number_of_reviews')
reviews_per_month_idx = headers.index('reviews_per_month')

reviews_col = data_filtered[:, reviews_idx].astype(float)
reviews_per_month_col = data_filtered[:, reviews_per_month_idx].astype(float)

has_reviews = (reviews_col > 0).astype(int)
review_frequency = np.where(reviews_col > 0, reviews_per_month_col, 0)

print(f"Tỷ lệ listings có reviews: {np.mean(has_reviews) * 100:.2f}%")
print(f"Review frequency trung bình: {np.mean(review_frequency):.4f}")

Tỷ lệ listings có reviews: 79.62%
Review frequency trung bình: 1.0948


In [13]:
room_type_idx = headers.index('room_type')
room_types = data_filtered[:, room_type_idx]

room_type_entire = (room_types == 'Entire home/apt').astype(int)
room_type_private = (room_types == 'Private room').astype(int)
room_type_shared = (room_types == 'Shared room').astype(int)

print(f"Entire home/apt: {np.sum(room_type_entire)}")
print(f"Private room: {np.sum(room_type_private)}")
print(f"Shared room: {np.sum(room_type_shared)}")

Entire home/apt: 25207
Private room: 22269
Shared room: 1155


In [14]:
neighbourhood_idx = headers.index('neighbourhood_group')
neighbourhoods = data_filtered[:, neighbourhood_idx]

neighbourhood_manhattan = (neighbourhoods == 'Manhattan').astype(int)
neighbourhood_brooklyn = (neighbourhoods == 'Brooklyn').astype(int)
neighbourhood_queens = (neighbourhoods == 'Queens').astype(int)
neighbourhood_bronx = (neighbourhoods == 'Bronx').astype(int)
neighbourhood_staten = (neighbourhoods == 'Staten Island').astype(int)

print(f"Manhattan: {np.sum(neighbourhood_manhattan)}")
print(f"Brooklyn: {np.sum(neighbourhood_brooklyn)}")
print(f"Queens: {np.sum(neighbourhood_queens)}")
print(f"Bronx: {np.sum(neighbourhood_bronx)}")
print(f"Staten Island: {np.sum(neighbourhood_staten)}")

Manhattan: 21482
Brooklyn: 20035
Queens: 5654
Bronx: 1089
Staten Island: 371


## 5. Tạo Dataset cho Modeling

In [15]:
latitude_idx = headers.index('latitude')
longitude_idx = headers.index('longitude')
availability_idx = headers.index('availability_365')
host_listings_idx = headers.index('calculated_host_listings_count')

latitude = data_filtered[:, latitude_idx].astype(float).reshape(-1, 1)
longitude = data_filtered[:, longitude_idx].astype(float).reshape(-1, 1)
minimum_nights = data_filtered[:, min_nights_idx].astype(float).reshape(-1, 1)
number_of_reviews = data_filtered[:, reviews_idx].astype(float).reshape(-1, 1)
availability = data_filtered[:, availability_idx].astype(float).reshape(-1, 1)
host_listings = data_filtered[:, host_listings_idx].astype(float).reshape(-1, 1)

X = np.hstack([
    latitude,
    longitude,
    room_type_entire.reshape(-1, 1),
    room_type_private.reshape(-1, 1),
    minimum_nights,
    number_of_reviews,
    availability,
    host_listings,
    neighbourhood_manhattan.reshape(-1, 1),
    neighbourhood_brooklyn.reshape(-1, 1),
    neighbourhood_queens.reshape(-1, 1),
    neighbourhood_bronx.reshape(-1, 1)
])

y = data_filtered[:, price_idx].astype(float)

feature_names = [
    'latitude', 'longitude', 'room_type_entire', 'room_type_private',
    'minimum_nights', 'number_of_reviews', 'availability_365',
    'host_listings_count', 'neighbourhood_manhattan', 'neighbourhood_brooklyn',
    'neighbourhood_queens', 'neighbourhood_bronx'
]

print(f"Shape của X: {X.shape}")
print(f"Shape của y: {y.shape}")
print(f"\nFeatures: {', '.join(feature_names)}")

Shape của X: (48631, 12)
Shape của y: (48631,)

Features: latitude, longitude, room_type_entire, room_type_private, minimum_nights, number_of_reviews, availability_365, host_listings_count, neighbourhood_manhattan, neighbourhood_brooklyn, neighbourhood_queens, neighbourhood_bronx


### Ý nghĩa Feature Engineering:

**One-Hot Encoding cho Categorical:**
- Room type và Neighbourhood group được chuyển thành binary features
- Cho phép mô hình học được sự khác biệt giữa các categories
- Tránh mô hình hiểu nhầm thành thứ tự (ordinal)

**Feature mới - Review Frequency:**
- Tính từ number_of_reviews và reviews_per_month
- Phản ánh mức độ hoạt động gần đây của listing
- Giúp mô hình hiểu được popularity và recency

**Lợi ích:**
- Tăng khả năng dự đoán của mô hình
- Capture được thông tin phi tuyến
- Giảm complexity nhờ chuẩn hóa dữ liệu

## 6. Chuẩn hóa Features

In [16]:
X_normalized = X.copy()

for i in range(X.shape[1]):
    col = X[:, i]
    min_val = np.min(col)
    max_val = np.max(col)
    
    if max_val - min_val > 0:
        X_normalized[:, i] = (col - min_val) / (max_val - min_val)

print("Thống kê sau chuẩn hóa:")
for i, name in enumerate(feature_names):
    print(f"{name:25} - Min: {np.min(X_normalized[:, i]):.4f}, Max: {np.max(X_normalized[:, i]):.4f}, Mean: {np.mean(X_normalized[:, i]):.4f}")

Thống kê sau chuẩn hóa:
latitude                  - Min: 0.0000, Max: 1.0000, Mean: 0.5545
longitude                 - Min: 0.0000, Max: 1.0000, Mean: 0.5502
room_type_entire          - Min: 0.0000, Max: 1.0000, Mean: 0.5183
room_type_private         - Min: 0.0000, Max: 1.0000, Mean: 0.4579
minimum_nights            - Min: 0.0000, Max: 1.0000, Mean: 0.0159
number_of_reviews         - Min: 0.0000, Max: 1.0000, Mean: 0.0371
availability_365          - Min: 0.0000, Max: 1.0000, Mean: 0.3079
host_listings_count       - Min: 0.0000, Max: 1.0000, Mean: 0.0189
neighbourhood_manhattan   - Min: 0.0000, Max: 1.0000, Mean: 0.4417
neighbourhood_brooklyn    - Min: 0.0000, Max: 1.0000, Mean: 0.4120
neighbourhood_queens      - Min: 0.0000, Max: 1.0000, Mean: 0.1163
neighbourhood_bronx       - Min: 0.0000, Max: 1.0000, Mean: 0.0224


## 7. Lưu Dữ liệu Đã Xử lý

In [18]:
processed_data = np.column_stack([X_normalized, y])
processed_headers = feature_names + ['price']

output_path = '../data/processed/processed_data.csv'
save_data(processed_data, processed_headers, output_path)

print(f"Đã lưu dữ liệu đã xử lý vào: {output_path}")
print(f"Kích thước: {processed_data.shape}")
print(f"Columns: {', '.join(processed_headers)}")

Đã lưu dữ liệu đã xử lý vào: ../data/processed/processed_data.csv
Kích thước: (48631, 13)
Columns: latitude, longitude, room_type_entire, room_type_private, minimum_nights, number_of_reviews, availability_365, host_listings_count, neighbourhood_manhattan, neighbourhood_brooklyn, neighbourhood_queens, neighbourhood_bronx, price
